In [1]:
import json
import os
import torch
from transformers import BertTokenizer, BertModel
import os.path as osp

/home/yiyao_yang/anaconda3/envs/graphgpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path = '/data/LPJ/ICML25/GraphCoder/test_mkdir/son_dir'
os.makedirs(folder_path, exist_ok=True)
print(f"已创建文件夹及其子文件夹：{folder_path}")


已创建文件夹及其子文件夹：/data/LPJ/ICML25/GraphCoder/test_mkdir/son_dir


In [2]:
bert_tokenizer = BertTokenizer.from_pretrained(
        '/data/LPJ/bert/bert-L12-H128-uncased',
        model_max_length=25,
        padding_side="right")


In [3]:
raw_node_attri = ['aaaaaa', 'i am your father and you are my son']

In [4]:
rslt = bert_tokenizer(
            raw_node_attri,
            return_tensors="pt",
            padding="longest",
            max_length=bert_tokenizer.model_max_length,
            truncation=True
        )

In [5]:
rslt

{'input_ids': tensor([[  101, 13360, 11057,  2050,   102,     0,     0,     0,     0,     0,
             0],
        [  101,  1045,  2572,  2115,  2269,  1998,  2017,  2024,  2026,  2365,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
**rslt

SyntaxError: invalid syntax (1093260329.py, line 1)

In [9]:
bert_tokenizer.pad_token
bert_tokenizer.pad_token_id
bert_tokenizer.eos_token
bert_tokenizer.eos_token_id


In [11]:
bert_model = BertModel.from_pretrained('/data/LPJ/bert/bert-L12-H128-uncased', torch_dtype=torch.bfloat16, device_map='cpu')


/home/yiyao_yang/anaconda3/envs/graphgpt/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
rep = bert_model(**rslt)


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.0312,  0.3008, -0.7188,  ...,  0.0064, -0.4648, -0.2490],
         [ 0.9688,  0.0889, -1.5938,  ..., -1.3750, -1.2656,  0.4629],
         [ 1.0312, -0.9570, -1.5781,  ..., -1.0781, -1.2734, -0.1660],
         ...,
         [-0.3438,  0.1875, -0.5508,  ...,  0.0659, -0.0747,  0.2158],
         [-0.1426,  0.1514, -0.8008,  ...,  0.0449, -0.1279,  0.2773],
         [ 0.6211,  0.3379, -1.4375,  ..., -1.1484, -1.0625,  0.2852]],

        [[-0.9727,  0.1641,  1.3516,  ..., -0.9727, -0.2363, -0.7422],
         [ 0.7383, -0.8555,  0.1631,  ..., -1.5000,  1.0312,  0.2041],
         [ 0.5195, -0.2754, -0.3008,  ..., -0.8789,  1.4922, -0.9648],
         ...,
         [ 0.5273, -1.1719,  0.5039,  ..., -0.2432,  0.2021, -0.3262],
         [-0.0352,  0.0500,  1.1016,  ..., -1.3906, -0.7852,  0.7070],
         [-1.1328, -0.5664,  0.3320,  ..., -0.3438, -0.3457, -0.6289]]],
       dtype=torch.bfloat16, grad_fn=<NativeLayerNorm

In [19]:
rep.pooler_output.shape

torch.Size([2, 128])